In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize    
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn't guaranteed
gpu = GPUs[0]
#gpu2 = GPUs[1]

def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print('GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB'.format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
#  print('GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB'.format(gpu2.memoryFree, gpu2.memoryUsed, gpu2.memoryUtil*100, gpu2.memoryTotal))
    
printm()

Tue May 11 12:15:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    59W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

In [2]:
%pip install transformers
%pip install datasets
import pandas as pd
import pyarrow
if int(pyarrow.__version__.split('.')[1]) < 16 and int(pyarrow.__version__.split('.')[0]) == 0:
    import os

    
    os.kill(os.getpid(), 9)
from datasets import load_dataset, Dataset
from datasets import ClassLabel, Features, Value
# Tensorflow classification with imbalanced data https://www.tensorflow.org/tutorials/structured_data/imbalanced_data#calculate_class_weights
# How to choose loss function https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/
#https://colab.research.google.com/drive/1kEg0SnYNtw_IJwu_kl5y3qRVs-BKBmNO#scrollTo=W64yQeCyl8sC
#https://github.com/NadirEM/nlp-notebooks/blob/master/Fine_tune_ALBERT_sentence_pair_classification.ipynb
#https://github.com/abhimishra91/transformers-tutorials/blob/master/transformers_multiclass_classification.ipynb
#https://huggingface.co/transformers/master/custom_datasets.html
#https://huggingface.co/transformers/custom_datasets.html#sequence-classification-with-imdb-reviews

  Using cached filelock-3.0.12-py3-none-any.whl (7.6 kB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [34]:
max_length=256
model_name='dbmdz/bert-base-german-cased'



In [4]:

from sklearn.model_selection import train_test_split

def load_data(file_path):
    loaded_data = pd.read_csv(file_path, sep='\t', header=None)
    return loaded_data


def reverse_multihot(val):
    encoding = {0:'none', 1:'exact', 2:'broader', 3:'narrower', 4:'related'}
    return encoding[val]

def split_data(data, testset_ratio):
    train_df, remaining = train_test_split(data, random_state=42, train_size=0.75, stratify=data.labels.values)
    return train_df, remaining
    #data = data.sample(frac=1)
    #f = int(len(data) * testset_ratio)
    #return data[f:], data[:f]

def multihot(val):
    encoding = {'none':0, 'exact':1, 'broader':2, 'narrower':3, 'related':4}
    return encoding[val]


def encode(data):
  return tokenizer(data['text_a'], data['text_b'], truncation=True, max_length=max_length, padding='max_length')



In [11]:

folder = 'german.tsv'
en_data = load_data(folder)
en_data = en_data[[2,3,4]]
en_data.columns =  ['text_a','text_b','labels']
en_data['labels'].value_counts()
#en_data['labels'] = en_data['labels'].map(lambda val: multihot(val))

test_file = 'german_reference.tsv'
test_data_orig = load_data(test_file)
test_data = test_data_orig[[2,3,4]]
test_data.columns = ['text_a', 'text_b', 'labels']

In [12]:
label_list = ['broader','narrower','exact','related','none']
en_data['labels'].apply(label_list.index)
test_data['labels'].apply(label_list.index)

0      0
1      4
2      4
3      4
4      0
      ..
564    4
565    0
566    4
567    2
568    4
Name: labels, Length: 569, dtype: int64

In [13]:
trainset, valset = split_data(en_data, 0.2)
trainset.shape, valset.shape
trainset.text_a.values

array(['Helligkeit', 'verheiratete Frau, Ehefrau',
       'die höchste Augenzahl auf einem Würfel', ...,
       'Maschine zum strecken|Strecken von Kardenband|Kardenbändern in der Textilindustrie',
       'Laut der Sprache, bei dem der Luftstrom relativ ungehindert durch den offenen Mund oder die Nase austritt und die Stimmbänder schwingen',
       'Aufstellung militärischer Einheiten'], dtype=object)

In [14]:
from sklearn import preprocessing
import numpy



In [15]:
le = preprocessing.LabelEncoder()
le.fit(en_data['labels'].to_list())

labels = le.classes_
labelMap = {v: k for v, k in enumerate(labels)}
    
print(labelMap)

def onehot_encode(data, encoder):

  integer_encoded = encoder.transform(data['labels'].to_list())
  onehot_encoder = preprocessing.OneHotEncoder(sparse=False)
  integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
  
  return onehot_encoder.fit_transform(integer_encoded)

{0: 'broader', 1: 'exact', 2: 'narrower', 3: 'none', 4: 'related'}


In [16]:
onehot_encoded_train = onehot_encode(trainset, le)
onehot_encoded_val = onehot_encode(valset, le)
onehot_encoded_test = onehot_encode(test_data, le)

In [17]:
val_y = le.inverse_transform([numpy.argmax(onehot_encoded_val[2, :])])
val_y


array(['none'], dtype='<U8')

In [18]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight
y_integers = np.argmax(onehot_encoded_train, axis=1)
class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
d_class_weights = dict(enumerate(class_weights))
sample_weights = compute_sample_weight('balanced', onehot_encoded_train)
sample_weights_val = compute_sample_weight('balanced', onehot_encoded_val)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass classes=[0 1 2 3 4], y=[3 1 3 ... 3 3 3] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [19]:
set(sample_weights)

{0.05361500022988054,
 1.7272945791805907,
 2.0803419016605824,
 2.3993672862712545,
 6.681576372868168}

In [20]:
train_a = trainset['text_a'].to_list()
train_b = trainset['text_b'].to_list()
train_labels = onehot_encoded_train

val_a = valset['text_a'].to_list()
val_b = valset['text_b'].to_list()
val_labels = onehot_encoded_val

test_a = test_data['text_a'].to_list()
test_b = test_data['text_b'].to_list()
test_labels = onehot_encoded_test

In [35]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained(model_name)



In [36]:
train_encodings = tokenizer(train_a, train_b, truncation=True, padding='longest',max_length=max_length, return_tensors='tf')
val_encodings = tokenizer(val_a, val_b, truncation=True, padding='longest', max_length=max_length, return_tensors='tf')
test_encodings = tokenizer(test_a, test_b, truncation=True,padding='longest', max_length=max_length,return_tensors='tf')

In [37]:

import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
    #sample_weights
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
    #sample_weights_val
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))
# Oversampling
# https://github.com/tensorflow/tensorflow/issues/14451
#https://keras.io/api/models/model_training_apis/  
# https://stackoverflow.com/questions/53653485/training-with-tf-data-api-and-sample-weights

In [38]:
import tensorflow_datasets as tfds
for pair, label in tfds.as_numpy(train_dataset.take(1)):
    print('pair', pair, label)

pair {'input_ids': array([  102, 10373,  1015,   103,  4246,   818,  8213,  1322,  3392,
         394, 11618,   186,   818,   556,  3903,  7893,   566,   103,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0], dtype=int32), 'token_type_ids': array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0

In [39]:
train_labels[0]
le.inverse_transform([0])
!pip install wandb
!wandb login 8d2052d6ffc37b0b4398c538a9604fe6b18e0cf9



wandb: Appending key for api.wandb.ai to your netrc file: /home/jupyter/.netrc


In [40]:
!pip install Keras==2.4.3
!pip install tensorflow-addons==0.8.3

In [41]:
import keras
#https://www.tensorflow.org/guide/keras/custom_callback#an_overview_of_callback_methods
class LossAndErrorPrintingCallback(keras.callbacks.Callback):
    def on_train_batch_end(self, batch, logs=None):
        print("For batch {}, loss is {:7.2f}.".format(batch, logs["loss"]))

    def on_test_batch_end(self, batch, logs=None):
        print("For batch {}, loss is {:7.2f}.".format(batch, logs["loss"]))

    def on_epoch_end(self, epoch, logs=None):
        print(
            "The average loss for epoch {} is {:7.2f} ".format(
                epoch, logs["loss"]
            )
        )

class LRLogger(keras.callbacks.Callback):
    """Learning rate scheduler which sets the learning rate according to schedule.

  Arguments:
      schedule: a function that takes an epoch index
          (integer, indexed from 0) and current learning rate
          as inputs and returns a new learning rate as output (float).
  """

    def __init__(self):
        super(LRLogger, self).__init__()

    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, "lr"):
            raise ValueError('Optimizer must have a "lr" attribute.')
        # Get the current learning rate from model's optimizer.
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        wandb.log({'epoch': epoch, 'learning_rate': lr})
        # Call schedule function to get the scheduled learning rate.
        #scheduled_lr = self.schedule(epoch, lr)
        # Set the value back to the optimizer before this epoch starts
        #tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
        #print("\nEpoch %05d: Learning rate is %6.4f." % (epoch, scheduled_lr))

In [42]:

from wandb.keras import WandbCallback

import wandb
import gc



#from transformers import TFDistilBertForSequenceClassification
from transformers import TFBertForSequenceClassification,AdamWeightDecay
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow_addons.metrics import F1Score
from tensorflow.keras.callbacks import ReduceLROnPlateau

seed=7
tf.random.set_seed(seed)

#wandb.config.globalseed=seed
#wandb.config.nr_gpus=len(GPU.getGPUs())
#wandb.config.lr = 4e-5
#wandb.config.weight_decay_rate=0.2
#wandb.config.epoch=10
#wandb.config.batch=32
#wandb.config.max_length=max_length
#wandb.config.lr_scheduler='ReduceLROnPlateau'
#wandb.config.seed=seed
#wandb.config.model=model_name






In [43]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=3, min_lr=0)

# Define multi-gpu strategy 
mirrored_strategy = tf.distribute.MirroredStrategy()
# Update batch size value
#wandb.config.batch *= mirrored_strategy.num_replicas_in_sync
# Create strategy scope to perform training

# Transformer and Batchsize
#https://dmitry.ai/t/topic/100/2


class RemoveGarbaseCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    gc.collect()



INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')


In [44]:
class_weights_dict={}
for i in range(len(class_weights)):
    class_weights_dict[i]=class_weights[i]
    

In [45]:
from transformers.optimization_tf import WarmUp
warmup_scheduler = WarmUp(1e-5, reduce_lr, warmup_steps=76)

In [46]:
def train():
    
    hyperparameter_defaults = dict(
        nr_gpus=len(GPU.getGPUs()),
        lr_scheduler=warmup_scheduler,
        model=model_name,
        globalseed=seed,
        weight_decay_rate=0.3,
        epoch=10,
        batch=32,
        max_length=max_length,
        seed=seed
    )
    wandb.init(project="mwsa_transformers_de", config=hyperparameter_defaults)
    config = wandb.config

        
    with mirrored_strategy.scope():
        #https://www.tensorflow.org/api_docs/python/tf/keras/losses/CategoricalCrossentropy
        #loss = {'issue': CategoricalCrossentropy(from_logits = False), 'product': CategoricalCrossentropy(from_logits = True)}
        loss = CategoricalCrossentropy(from_logits=True)
        metric = [CategoricalAccuracy(), F1Score(num_classes=5, average='weighted')]
        # How to choose loss functions https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/
        #https://stackoverflow.com/questions/59305514/tensorflow-how-to-use-tf-keras-metrics-in-multiclass-classification
        #https://towardsdatascience.com/keras-accuracy-metrics-8572eb479ec7#:~:text=Categorical%20Accuracy%20calculates%20the%20percentage,yTrue%2C%20it%20is%20considered%20accurate.
        #optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
        #optimizer = AdamWeightDecay(learning_rate=0.01, weight_decay_rate=0.1)#silvery-plasma-24
        optimizer = AdamWeightDecay(learning_rate=1e-5, weight_decay_rate=wandb.config.weight_decay_rate)

        model = TFBertForSequenceClassification.from_pretrained(model_name,num_labels=5, return_dict=True)
        #model.summary()
        model.config.gradient_checkpointing=True

        #https://keras.io/api/models/model_training_apis/
        model.compile(optimizer=optimizer, loss=loss, metrics=metric, weighted_metrics=metric) # can also use any keras loss fn
        history = model.fit(train_dataset.shuffle(1000,seed=wandb.config.seed).batch(wandb.config.batch), 
              epochs=wandb.config.epoch,
              validation_data=val_dataset.shuffle(1000,seed=wandb.config.seed).batch(wandb.config.batch), 
                            class_weight=class_weights_dict,
                            callbacks=[WandbCallback(),reduce_lr,LRLogger(),RemoveGarbaseCallback()])
        
        return model



# Steps per epoch
# https://datascience.stackexchange.com/questions/47405/what-to-set-in-steps-per-epoch-in-keras-fit-generator

#https://www.tensorflow.org/tutorials/keras/text_classification
# Increasing Loss and accuracy
#Overfitting
#https://datascience.stackexchange.com/questions/76527/overfitting-in-huggingfaces-tfbertforsequenceclassification

In [47]:
sweep_config = {
  "name": "MWSA Sweep",
  "method": "grid",
  "parameters": {
        "lr": {
            "values": [5e-5,5e-4,5e-6]
        },
        "weight_decay_rate":{
            "values": [0.1,0.2,0.3]
        },
        "batch": {
            "values": [32]
        }
  }
}

sweep_id = wandb.sweep(sweep_config)

model = train()
#wandb.agent(sweep_id, train)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: jzr3r2yi
Sweep URL: https://wandb.ai/sbyim/uncategorized/sweeps/jzr3r2yi


wandb: wandb version 0.10.30 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-german-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
INFO:tensorflow:batch_all_reduce: 198 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replic

wandb: ERROR Can't save model, h5py returned error: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.


Epoch 2/10
104/104 [==============================] - 81s 776ms/step - loss: 1.3357 - weighted_categorical_accuracy: 0.5475 - weighted_f1_score: 0.6904 - val_loss: 0.8598 - val_weighted_categorical_accuracy: 0.6954 - val_weighted_f1_score: 0.7232
Epoch 3/10
104/104 [==============================] - 80s 769ms/step - loss: 1.2024 - weighted_categorical_accuracy: 0.6099 - weighted_f1_score: 0.7286 - val_loss: 0.9046 - val_weighted_categorical_accuracy: 0.6881 - val_weighted_f1_score: 0.7322
Epoch 4/10
104/104 [==============================] - 81s 775ms/step - loss: 0.9780 - weighted_categorical_accuracy: 0.7042 - weighted_f1_score: 0.7773 - val_loss: 0.7899 - val_weighted_categorical_accuracy: 0.7180 - val_weighted_f1_score: 0.7471
Epoch 5/10
104/104 [==============================] - 81s 775ms/step - loss: 0.7885 - weighted_categorical_accuracy: 0.7651 - weighted_f1_score: 0.8178 - val_loss: 0.8782 - val_weighted_categorical_accuracy: 0.6963 - val_weighted_f1_score: 0.7403
Epoch 6/10
1

In [48]:
model.config.id2label=labelMap

In [50]:
model.summary()
test_result = model.evaluate(test_dataset.batch(32))
print(model.metrics_names)
test_result

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109927680 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3845      
Total params: 109,931,525
Trainable params: 109,931,525
Non-trainable params: 0
_________________________________________________________________
18/18 [==============================] - 33s 153ms/step - loss: 0.8686 - weighted_categorical_accuracy: 0.7241 - weighted_f1_score: 0.7517
['loss', 'weighted_categorical_accuracy', 'weighted_f1_score', 'weighted_categorical_accuracy', 'weighted_f1_score']


[0.8686318397521973,
 0.7240773439407349,
 0.7517008781433105,
 0.7240773439407349,
 0.7517008781433105]

In [1]:
import os
save_path = os.path.join("mwsa","mwsa_best_de")




In [2]:
model.summary()
model.save_pretrained(save_path, saved_model=True)
tokenizer.save_pretrained(save_path)

NameError: name 'model' is not defined

In [3]:
# In order to use GCP for trained models, we have to save the model as TF Saved_Model format.
# Just calling model.save() doesn't work.
# https://colab.research.google.com/drive/1p1Nifh1P-vqAZ1gHsNSCXAHzVWzl5YPP
# https://github.com/huggingface/transformers/issues/2021
import os 

from transformers import TFBertForSequenceClassification
loaded_model = TFBertForSequenceClassification.from_pretrained(save_path)



All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at mwsa/mwsa_best_de.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [10]:
from transformers import BertTokenizerFast
loaded_tokenizer = BertTokenizerFast.from_pretrained(save_path)
input_encodings = loaded_tokenizer(["only me"],["only me"], max_length=256, truncation=True, padding='max_length', return_tensors="tf")
data = tf.data.Dataset.from_tensor_slices((
    dict(input_encodings)
))

print(data)

NameError: name 'tf' is not defined

In [8]:
input_encodings

{'input_ids': <tf.Tensor: shape=(1, 256), dtype=int32, numpy=
array([[  102, 21692,  6501,   103, 21692,  6501,   103,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0, 

In [9]:

#predict_input = (test_encodings,  truncation=True, max_length=128, padding='max_length')
result = loaded_model.predict(test_dataset.batch(32))
probabilities = tf.nn.softmax(result.logits)


NameError: name 'test_dataset' is not defined

In [55]:
labelMap

{0: 'broader', 1: 'exact', 2: 'narrower', 3: 'none', 4: 'related'}

In [56]:
#import sys
#import numpy
!pip install scikit-learn
import sklearn
from sklearn.metrics import classification_report
numpy.set_printoptions(threshold=200)

preds = tf.argmax(result.logits, axis=1).numpy()
print(classification_report(np.argmax(test_labels,axis=1),preds))

              precision    recall  f1-score   support

           0       0.23      0.51      0.32        43
           1       0.41      0.52      0.46        33
           2       0.20      0.31      0.24        29
           3       0.95      0.82      0.88       444
           4       0.17      0.05      0.08        20

    accuracy                           0.72       569
   macro avg       0.39      0.44      0.40       569
weighted avg       0.80      0.72      0.75       569



In [57]:
result_df = pd.Series(preds)
result_df = result_df.map(lambda res: le.inverse_transform([res])[0])

In [58]:
test_data_orig[4]=result_df.values

In [60]:
test_data_orig.to_csv("german/bert_german_20210511", sep='\t', index = False)